In [3]:
import hmac
import json
import time
import requests
import hashlib
import requests
import pandas as pd
from datetime import datetime, timedelta
from pandas import json_normalize
pd.set_option('display.max_columns', None)
from datetime import datetime, timedelta
import asyncio
import aiohttp 
from tqdm.notebook import tqdm
from concurrent.futures import ThreadPoolExecutor
import concurrent.futures

## 1. Get and refresh token

### A. Create date table

In [4]:
from datetime import datetime, timedelta

# Define start and end dates
date_from = '2021-01-01'
date_to = '2023-12-31'

# Convert date strings to datetime objects
start_date = datetime.strptime(date_from, '%Y-%m-%d')
end_date = datetime.strptime(date_to, '%Y-%m-%d')

# Generate a list of dates between start and end dates (inclusive)
date_list = [start_date + timedelta(days=x) for x in range((end_date - start_date).days + 1)]

# Convert the dates back to string format if needed
date_list = [date.strftime('%Y-%m-%d') for date in date_list]

# Print the list of dates
date_list[0:5]

['2021-01-01', '2021-01-02', '2021-01-03', '2021-01-04', '2021-01-05']

### B. Refresh token and save to file

#### B1. All variables

In [5]:
access_token = 'tokens_crv'
shop_id = 123
list_path = "order_lst_crv.json"

partner_id = 123
tmp_partner_key = "123"
file_path = r"C:\Users\gianhubinh.le\Jupyter lab\API automation - Shopee\Access token\{}.json".format(access_token)

#### B2. Refresh token function

In [ ]:
with open(file_path, 'r') as file:
    tokens_dict = json.load(file)
refresh_token = tokens_dict['refresh_token']


def get_token_shop_level():

    with open(file_path, 'r') as file:
        tokens_dict = json.load(file)
    refresh_token = tokens_dict['refresh_token']

    timest = int(time.time())
    host = "https://partner.shopeemobile.com"
    path = "/api/v2/auth/access_token/get"
    body = {"shop_id": shop_id, "partner_id": partner_id, "refresh_token": refresh_token}
    tmp_base_string = "%s%s%s" % (partner_id, path, timest)
    base_string = tmp_base_string.encode()
    partner_key = tmp_partner_key.encode()
    sign = hmac.new(partner_key, base_string, hashlib.sha256).hexdigest()
    url = host + path + "?partner_id=%s&timestamp=%s&sign=%s" % (partner_id, timest, sign)
    # print(url)
    headers = {"Content-Type": "application/json"}
    resp = requests.post(url, json=body, headers=headers)
    ret = json.loads(resp.content)
    access_token = ret.get("access_token")
    new_refresh_token = ret.get("refresh_token")
    print(access_token, new_refresh_token)
    
    # Create a dictionary to store the tokens
    tokens_dict = {
        "access_token": access_token,
        "refresh_token": new_refresh_token
    }
    
    # Write the tokens to the JSON file
    with open(file_path, 'w') as file:
        json.dump(tokens_dict, file)
    
    return access_token, new_refresh_token

access_token, new_refresh_token = get_token_shop_level()

## 2. Get the order list

### A. Get Order intraday function

In [ ]:
order_lst_total = []

def get_order_lst(time_from, time_to, cursor):

    # get access token
    with open(file_path, 'r') as file:
        tokens_dict = json.load(file)
        access_token = tokens_dict['access_token']

    def sign_generate(access_token, path):
        partner_key = tmp.encode()
        path = path
        timest = int(time.time())
        tmp_base_string = "%s%s%s%s%s"%(partner_id, path, timest, access_token, shop_id) 
        base_string = tmp_base_string.encode()
        sign = hmac.new(partner_key, base_string, hashlib.sha256).hexdigest()
        return sign
    
    sign = sign_generate(access_token, '/api/v2/order/get_order_list')
    
    params = {
        'access_token': access_token,
        'page_size': '100',
        'partner_id': '123',
        'shop_id': str(shop_id),
        'sign': sign,
        'time_from': time_from,
        'time_range_field': 'create_time',
        'time_to': time_to,
        'timestamp': int(time.time()),
        'cursor': cursor
    }

    response = requests.get('https://partner.shopeemobile.com/api/v2/order/get_order_list', params=params)
    try:
        data = response.json()
        return data
    except exception as e:
        print(e)

def get_order_intraday(time_from, time_to):
    cursor = 0
    while True:
        try:
            # data = get_order_lst(1701450000, 1701536399, cursor)
            # print(cursor)
            data = get_order_lst(time_from, time_to, cursor)
            order_sn_list = [order['order_sn'] for order in data['response']['order_list']]
            order_lst_total.extend(order_sn_list)
            
            # 1. There is still more data
            if data['response']['more'] == True:
                cursor += 100
            # 2. There is no more data
            if data['response']['more'] == False:
                break
        except Exception as e:
            print(f"An unexpected exception occurred: {e}")
            print(data)
            continue

get_order_lst(1701450000, 1701536399, 0)['response']['order_list'][0:5]

### B. Using thread to scrape a number of day

In [ ]:
order_lst_total = []

# Number of threads
if 1 <= len(date_list) <= 15:
    num_threads = len(date_list)
else:
    num_threads = 15

# Create a tqdm progress bar for the loop
with tqdm(total=len(date_list), unit="date") as progress_bar:

    # Divide dates into chunks for each thread
    chunk_size = len(date_list) // num_threads
    date_chunks = [date_list[i:i + chunk_size] for i in range(0, len(date_list), chunk_size)]

    # Function to process dates in a chunk
    def process_chunk(chunk):
        for date_string in chunk:
            # 1. get start and end date in epoch
            date = datetime.strptime(date_string, '%Y-%m-%d')
            time_from = int((date.replace(hour=0, minute=0, second=0)).timestamp())
            time_to = int((date.replace(hour=23, minute=59, second=59)).timestamp())
            # 2. running the function (replace this with your actual function)
            try:
                get_order_intraday(time_from, time_to)
                progress_bar.update(1)
            except Exception as e:
                print(f"An unexpected exception occurred: {e}")
                
            # print('Done ' + str(date))

    # Create a ThreadPoolExecutor with the specified number of threads
    with ThreadPoolExecutor(max_workers=num_threads) as executor:
        # Submit each chunk for processing
        futures = [executor.submit(process_chunk, chunk) for chunk in date_chunks]

        # Wait for all threads to complete
        concurrent.futures.wait(futures)

# order_lst_sample
print(len(order_lst_total))

# 2. Save the list to a JSON file
with open(list_path, 'w') as file:
    json.dump(order_lst_total, file)

# 3. Read the list
with open(list_path, 'r') as file:
    order_lst_total = json.load(file)

## 3. Get the order detail

### A. Get Order detail function

In [7]:
def get_order_detail(order_sn):

    # get access token
    with open(file_path, 'r') as file:
        tokens_dict = json.load(file)
        access_token = tokens_dict['access_token']
        
    def sign_generate(access_token, path):
        partner_key = tmp.encode()
        path = path
        timest = int(time.time())
        access_token = access_token
        tmp_base_string = "%s%s%s%s%s"%(partner_id, path, timest, access_token, shop_id) 
        base_string = tmp_base_string.encode()
        sign = hmac.new(partner_key, base_string, hashlib.sha256).hexdigest()
        return sign
    
    sign = sign_generate(access_token, '/api/v2/order/get_order_detail')
    
    params = {
        'access_token': access_token,
        'partner_id': '123',
        'shop_id': str(shop_id),
        'sign': sign,
        'timestamp': int(time.time()),
        'order_sn_list': order_sn, 
        'response_optional_fields': "buyer_user_id,buyer_username,estimated_shipping_fee,recipient_address,actual_shipping_fee ,goods_to_declare,note,note_update_time,item_list,pay_time,dropshipper,dropshipper_phone,split_up,buyer_cancel_reason,cancel_by,cancel_reason,actual_shipping_fee_confirmed,buyer_cpf_id,fulfillment_flag,pickup_done_time,package_list,shipping_carrier,payment_method,total_amount,buyer_username,invoice_data, checkout_shipping_carrier, reverse_shipping_fee, order_chargeable_weight_gram, edt, prescription_images, prescription_check_status"
    }
    
    
    response = requests.get('https://partner.shopeemobile.com/api/v2/order/get_order_detail', params=params)
    data = response.json()
    try:
        json_data = data['response']['order_list'][0]
        df_items = json_normalize(json_data['item_list'])
        del json_data['item_list']
        df = pd.DataFrame([json_data])
        # Create a dummy key for cross join
        df['key'] = 1
        df_items['key'] = 1
        # Perform a cross join (merge with the dummy key)
        df_expanded = pd.merge(df, df_items, on='key').drop('key', axis=1)
        return df_expanded
    except Exception as e:
        print('-----------------------1. Error get_order_detail(order_sn)----------------------------')
        print("Error message withint the get_order_detail(order_sn) function " + str(e))
        print(data)
        print('-----------------------------------------------------------------------------------')
        raise Exception
        
data = get_order_detail('123')
data

,actual_shipping_fee_confirmed,buyer_cancel_reason,buyer_cpf_id,buyer_user_id,buyer_username,cancel_by,cancel_reason,cod,create_time,currency,days_to_ship,dropshipper,dropshipper_phone,estimated_shipping_fee,fulfillment_flag,goods_to_declare,invoice_data,message_to_seller,note,note_update_time,order_sn,order_status,package_list,pay_time,payment_method,pickup_done_time,recipient_address,region,reverse_shipping_fee,ship_by_date,shipping_carrier,split_up,total_amount,update_time,item_id,item_name,item_sku,model_id,model_name,model_sku,model_quantity_purchased,model_original_price,model_discounted_price,wholesale,weight,add_on_deal,main_item,add_on_deal_id,promotion_type,promotion_id,order_item_id,promotion_group_id,product_location_id,is_prescription_item,is_b2c_owned_item,image_info.image_url
0,True,,None,None,****,,,False,1701536321,VND,2,None,None,32700,fulfilled_by_shopee,False,None,,,0,2312036UB2HVJ3,COMPLETED,"[{'package_number': 'OFG155235521253241', 'gro...",1701536333,ShopeePay Linked Bank Account,1701595116,"{'name': '****', 'phone': '****', 'town': '***...",VN,0,1701795536,SPX Express,False,151000,1701869841,21134563727,Sữa rửa mặt giúp sạch sâu dành cho da dầu Cera...,3337875597340,0,,,1,155000,153000,False,0.2,True,True,134352441250179,add_on_free_gift_main,134352441250179,21134563727,134352441250179,[VNS],False,False,https://cf.shopee.vn/file/vn-11134207-7r98o-lo...


### B. Using asyncio to scrape multiple orders

In [ ]:
dict_dtl_lst = []

# Read the list
with open(list_path, 'r') as file:
    order_lst_total = json.load(file)

# Divide order_lst_total into parts
num_chunks = 50
chunk_size = len(order_lst_total) // num_chunks
order_chunks = [order_lst_total[i:i + chunk_size] for i in range(0, len(order_lst_total), chunk_size)]

# Set limit for asyncio code
semaphore_limit = 200  # Set your desired limit here
semaphore = asyncio.Semaphore(semaphore_limit)
        
def get_access_token():
    with open(file_path, 'r') as file:
        tokens_dict = json.load(file)
        # print('access_token', tokens_dict)
        return tokens_dict['access_token']

def sign_generate(access_token, path):
    partner_id = 123
    partner_key = tmp.encode()
    path = path
    timest = int(time.time())
    access_token = access_token
    tmp_base_string = "%s%s%s%s%s"%(partner_id, path, timest, access_token, shop_id) 
    base_string = tmp_base_string.encode()
    sign = hmac.new(partner_key, base_string, hashlib.sha256).hexdigest()
    return sign

def convert_df_to_dict(df, dict_dtl_lst_chunk):
    for row in df.to_dict(orient='records'):
        dict_dtl_lst_chunk.append(row)
        
async def get_order_data(session, order_sn, total_orders, dict_dtl_lst_chunk):
    
    access_token = get_access_token()

    sign = sign_generate(access_token, '/api/v2/order/get_order_detail')
            
    params = {
        'access_token': access_token,
        'partner_id': '123',
        'shop_id': str(shop_id),
        'sign': sign,
        'timestamp': int(time.time()),
        'order_sn_list': order_sn, 
        'response_optional_fields': "buyer_user_id,buyer_username,estimated_shipping_fee,recipient_address,actual_shipping_fee ,goods_to_declare,note,note_update_time,item_list,pay_time,dropshipper,dropshipper_phone,split_up,buyer_cancel_reason,cancel_by,cancel_reason,actual_shipping_fee_confirmed,buyer_cpf_id,fulfillment_flag,pickup_done_time,package_list,shipping_carrier,payment_method,total_amount,buyer_username,invoice_data, checkout_shipping_carrier, reverse_shipping_fee, order_chargeable_weight_gram, edt, prescription_images, prescription_check_status"
    }

    async with semaphore, session.get('https://partner.shopeemobile.com/api/v2/order/get_order_detail', params=params) as response:
        try:
            data = await response.json()
            json_data = data['response']['order_list'][0]
            df_items = json_normalize(json_data['item_list'])
            del json_data['item_list']
            df = pd.DataFrame([json_data])
            # Create a dummy key for cross join
            df['key'] = 1
            df_items['key'] = 1
            # Perform a cross join (merge with the dummy key)
            df_expanded = pd.merge(df, df_items, on='key').drop('key', axis=1)
            # append result
            convert_df_to_dict(df_expanded, dict_dtl_lst_chunk)
            print(order_sn, end='\r')
            
        except Exception as e:
            print(data, order_sn)
            print(e)
            raise e

async def main():
    async with  aiohttp.ClientSession() as session:
        i = 0
        while i < len(order_chunks):
            try:
                print(str(i+1) + '/' + str(num_chunks))
                get_token_shop_level()
                time.sleep(2)
                total_orders = len(order_lst_total)
                dict_dtl_lst_chunk = []
                coroutines = [get_order_data(session, order, total_orders, dict_dtl_lst_chunk) for order in order_chunks[i]]
                await asyncio.gather(*coroutines)
                dict_dtl_lst.extend(dict_dtl_lst_chunk)
                i += 1
            except Exception as e:
                time.sleep(30)
                continue
                # raise e

await main()

### C. Combine all rows & transform to the correct data type

In [ ]:
df_result = pd.DataFrame(dict_dtl_lst)
    
df_result = df_result.rename(columns={'image_info.image_url': 'image_url'})
df_result['package_list'] = df_result['package_list'].astype(str)
df_result['pay_time'].fillna(0, inplace=True)
df_result['pay_time'] = df_result['pay_time'].astype('int64')
df_result['recipient_address'] = df_result['recipient_address'].astype(str)
df_result['product_location_id'] = df_result['product_location_id'].astype(str)

print('Total items: ' + str(len(dict_dtl_lst)))
df_result.head(2)

### D. Load to GCP

In [34]:
import pandas_gbq

# TODO: Set project_id to your Google Cloud Platform project ID.
project_id = "spmena-sandbox-vn-apac-dv"

# TODO: Set table_id to the full destination table ID (including the dataset ID).
table_id = 'vn_api_project.shopee_oms_crv_order_detail'

pandas_gbq.to_gbq(df_result, table_id, project_id=project_id, if_exists = 'replace')

C:\Users\gianhubinh.le\AppData\Local\anaconda3\Lib\site-packages\google\auth\_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
100%|████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<?, ?it/s]


## 4. Get the discount detail

### A. Get Order detail function

In [36]:
def get_discount(discount_id):

    path = '/api/v2/payment/get_escrow_detail'
    
    # get access token
    with open(file_path, 'r') as file:
        tokens_dict = json.load(file)
        access_token = tokens_dict['access_token']
        
    def sign_generate(access_token, path):
        partner_key = tmp.encode()
        path = path
        timest = int(time.time())
        access_token = access_token
        tmp_base_string = "%s%s%s%s%s"%(partner_id, path, timest, access_token, shop_id) 
        base_string = tmp_base_string.encode()
        sign = hmac.new(partner_key, base_string, hashlib.sha256).hexdigest()
        return sign
    
    sign = sign_generate(access_token, path)
    
    params = {
        'access_token': access_token,
        'partner_id': '123',
        'shop_id': str(shop_id),
        'sign': sign,
        'timestamp': int(time.time()),
        'order_sn': discount_id}
    
    response = requests.get('https://partner.shopeemobile.com' + path, params=params)
    data = response.json()
    json_data = data['response']['order_income']
    df_items = json_normalize(json_data['items'])
    del json_data['items']
    df = pd.DataFrame([json_data])
    # Create a dummy key for cross join
    df['key'] = 1
    df_items['key'] = 1
    # Perform a cross join (merge with the dummy key)
    df_expanded = pd.merge(df, df_items, on='key').drop('key', axis=1)
    df_expanded['order_sn'] = data['response']['order_sn']
    return df_expanded

data = get_discount(order_lst_total[0])
data

,actual_shipping_fee,buyer_paid_shipping_fee,buyer_payment_method,buyer_total_amount,buyer_transaction_fee,campaign_fee,coins,commission_fee,cost_of_goods_sold,credit_card_promotion,credit_card_transaction_fee,cross_border_tax,delivery_seller_protection_fee_premium_amount,drc_adjustable_refund,escrow_amount,escrow_amount_after_adjustment,escrow_tax,estimated_shipping_fee,final_escrow_product_gst,final_escrow_shipping_gst,final_product_protection,final_product_vat_tax,final_return_to_seller_shipping_fee,final_shipping_fee,final_shipping_vat_tax,fsf_seller_protection_fee_claim_amount,instalment_plan,order_ams_commission_fee,order_chargeable_weight,order_discounted_price,order_original_price,order_seller_discount,order_selling_price,original_cost_of_goods_sold,original_price_x,original_shopee_discount,payment_promotion,prorated_coins_value_offset_return_items,prorated_shopee_voucher_offset_return_items,reverse_shipping_fee,rsf_seller_protection_fee_claim_amount,sales_tax_on_lvg,seller_coin_cash_back,seller_discount_x,seller_lost_compensation,seller_return_refund,seller_shipping_discount,seller_transaction_fee,seller_voucher_code,service_fee,shipping_fee_discount_from_3pl,shipping_seller_protection_fee_amount,shopee_discount_x,shopee_shipping_rebate,total_adjustment_amount,voucher_from_seller,voucher_from_shopee,activity_id,activity_type,ams_commission_fee,discount_from_coin,discount_from_voucher_seller,discount_from_voucher_shopee,discounted_price,is_b2c_shop_item,is_main_item,item_id,item_name,item_sku,model_id,model_name,model_sku,original_price_y,quantity_purchased,seller_discount_y,selling_price,shopee_discount_y,order_sn
0,12800,6500,Credit Card/Debit Card,365500,0,0,0,17950,365000,0,9138,0,0,0,331912,331912,0,12800,0,0,0,0,0,-6500,0,0,N/A,0,0,365000,10380000,10015000,365000,365000,10380000,0,0,0,0,0,0,0,0,10015000,0,0,0,9138,[CERA116K],0,0,0,0,6300,0,6000,0,203008508,add_on_deal,0,0,0,0,0,False,False,18734562676,[HB GIFT HÀNG TẶNG KHÔNG BÁN] Sữa dưỡng giúp d...,3337875677325,0,,,10000000,1,10000000,0,0,2301028F5XVPSQ
1,12800,6500,Credit Card/Debit Card,365500,0,0,0,17950,365000,0,9138,0,0,0,331912,331912,0,12800,0,0,0,0,0,-6500,0,0,N/A,0,0,365000,10380000,10015000,365000,365000,10380000,0,0,0,0,0,0,0,0,10015000,0,0,0,9138,[CERA116K],0,0,0,0,6300,0,6000,0,203008508,add_on_deal,0,0,6000,0,365000,False,True,20832327452,[Mã COSACD6 giảm 10% đơn 400K] Sữa rửa mặt giú...,3337875597333,0,,,380000,1,15000,365000,0,2301028F5XVPSQ


### B. Using thread to scrape multiple orders

In [ ]:
dict_pmt_lst = []

# Read the list
with open(list_path, 'r') as file:
    order_lst_total = json.load(file)

# Divide order_lst_total into parts
num_chunks = 50
chunk_size = len(order_lst_total) // num_chunks
order_chunks = [order_lst_total[i:i + chunk_size] for i in range(0, len(order_lst_total), chunk_size)]

# Set limit for asyncio code
semaphore_limit = 20  # Set your desired limit here
semaphore = asyncio.Semaphore(semaphore_limit)
        
def get_access_token():
    with open(file_path, 'r') as file:
        tokens_dict = json.load(file)
        # print('access_token', tokens_dict)
        return tokens_dict['access_token']

def sign_generate(access_token, path):
    partner_key = tmp.encode()
    path = path
    timest = int(time.time())
    access_token = access_token
    tmp_base_string = "%s%s%s%s%s"%(partner_id, path, timest, access_token, shop_id) 
    base_string = tmp_base_string.encode()
    sign = hmac.new(partner_key, base_string, hashlib.sha256).hexdigest()
    return sign

def convert_df_to_dict(df, dict_pmt_lst_chunk):
    for row in df.to_dict(orient='records'):
        dict_pmt_lst_chunk.append(row)
        
async def get_order_data(session, order_sn, total_orders, dict_pmt_lst_chunk):
    
    access_token = get_access_token()

    sign = sign_generate(access_token, '/api/v2/payment/get_escrow_detail')
            
    params = {
        'access_token': access_token,
        'partner_id': '123',
        'shop_id': str(shop_id),
        'sign': sign,
        'timestamp': int(time.time()),
        'order_sn': order_sn
    }

    async with semaphore, session.get('https://partner.shopeemobile.com/api/v2/payment/get_escrow_detail', params=params) as response:
        try:
            data = await response.json()
            json_data = data['response']['order_income']
            df_items = json_normalize(json_data['items'])
            del json_data['items']
            df = pd.DataFrame([json_data])
            # Create a dummy key for cross join
            df['key'] = 1
            df_items['key'] = 1
            # Perform a cross join (merge with the dummy key)
            df_expanded = pd.merge(df, df_items, on='key').drop('key', axis=1)
            df_expanded['order_sn'] = data['response']['order_sn']
            # append result
            convert_df_to_dict(df_expanded, dict_pmt_lst_chunk)
            print(order_sn, end='\r')
            
        except Exception as e:
            print(data)
            print(e)
            raise e

async def main():
    async with  aiohttp.ClientSession() as session:
        i = 0
        while i < len(order_chunks):
            try:
                print(str(i+1) + '/' + str(num_chunks))
                get_token_shop_level()
                time.sleep(2)
                total_orders = len(order_lst_total)
                dict_pmt_lst_chunk = []
                coroutines = [get_order_data(session, order, total_orders, dict_pmt_lst_chunk) for order in order_chunks[i]]
                await asyncio.gather(*coroutines)
                dict_pmt_lst.extend(dict_pmt_lst_chunk)
                i += 1
            except Exception as e:
                time.sleep(60)
                continue
                # raise e

await main()

### C. Combine all rows & transform to the correct data type

In [ ]:
df_result = pd.DataFrame(dict_pmt_lst)
df_result['seller_voucher_code'] = df_result['seller_voucher_code'].astype(str)
df_result['model_sku'] = df_result['model_sku'].astype(str)
df_result['order_sn'] = df_result['order_sn'].astype(str)
df_result['order_adjustment'] = df_result['order_adjustment'].astype(str)

if 'is_b2c_shop_item' in df_result.columns:
    is_b2c_shop_item_column = df_result.pop('is_b2c_shop_item')
    df_result['is_b2c_shop_item'] = is_b2c_shop_item_column
    df_result['is_b2c_shop_item'] = df_result['is_b2c_shop_item'].astype(str)
else:
    df_result['is_b2c_shop_item'] = None
    df_result['is_b2c_shop_item'] = df_result['is_b2c_shop_item'].astype(str)
    
df_result['is_main_item'] = df_result['is_main_item'].astype(str)

print(len(df_result))
df_result.head(2)

### D. Load to GCP

In [6]:
import pandas_gbq

# TODO: Set project_id to your Google Cloud Platform project ID.
project_id = "spmena-sandbox-vn-apac-dv"

# TODO: Set table_id to the full destination table ID (including the dataset ID).
table_id = 'vn_api_project.shopee_oms_crv_order_payment'

pandas_gbq.to_gbq(df_result, table_id, project_id=project_id, if_exists = 'replace')

C:\Users\gianhubinh.le\AppData\Local\anaconda3\Lib\site-packages\google\auth\_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
100%|████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<?, ?it/s]
